# MCP Tools Deep Dive

This notebook explores the **Model Context Protocol (MCP)** and the 3 tools in our support system.

## Learning Objectives

- Understand what MCP is and why it exists
- Learn how to connect to an MCP server
- Test each tool and understand its behavior
- See how search algorithms work
- Modify tool parameters and observe results

---

**Prerequisites:** MCP server must be running:
```bash
python -m support_mcp_server.server
```

In [7]:
# Imports
import json
import asyncio
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.syntax import Syntax

from support_copilot_host.mcp_client import SupportMCPClient

console = Console()
console.print("[bold green]✓ Ready to explore MCP![/bold green]")

✓ Ready to explore MCP!

## What is MCP?

**Model Context Protocol** is a standardized way for AI applications to interact with tools/data sources.

### Before MCP
```
App A → Custom Integration → Database
App B → Different Integration → Same Database
App C → Yet Another Integration → Same Database
```
Every app reinvents the wheel!

### With MCP
```
Apps → MCP Client → MCP Server → Tools
                        ├── Docs Search
                        ├── Incident Search
                        └── Status Check
```
One standard protocol for all tools!

## 1. Connect to MCP Server

In [8]:
# Create client and connect
client = SupportMCPClient()

await client.connect()

console.print("[bold green]✓ Connected to MCP server![/bold green]")
console.print("[dim]The server is now ready to receive tool calls[/dim]")

✓ Connected to MCP server!

The server is now ready to receive tool calls

## 2. Tool #1: support_docs.search

Searches internal runbooks and documentation.

### How it works:
1. Loads all `.md` files from `data/docs/runbooks/`
2. Tokenizes the search query
3. Counts token occurrences in each document
4. Returns top N matches by score

In [9]:
# Simple search
results = await client.call_support_docs_search(
    query="export csv timeout",
    max_results=3
)

console.print(f"\n[bold cyan]Found {len(results)} results for 'export csv timeout'[/bold cyan]\n")

for i, doc in enumerate(results, 1):
    console.print(Panel(
        f"[yellow]Score: {doc['score']}[/yellow]\n\n"
        f"{doc['snippet']}",
        title=f"[bold]{i}. {doc['title']}[/bold]",
        subtitle=f"Path: {doc['path']}",
        border_style="cyan"
    ))
    console.print()

Found 2 results for 'export csv timeout'

╭──────────────────────────────────────────── 1. Export to CSV Errors ────────────────────────────────────────────╮
│ Score: 15                                                                                                       │
│                                                                                                                 │
│ # Export to CSV Errors                                                                                          │
│                                                                                                                 │
│ When users attempt to export reports to CSV and see errors like                                                 │
│ **"Export failed: timeout"** or **"Export failed: invalid workspace"**, follow                                  │
│ these steps:                                                                                                    │
│                                                                                                                 │
│ 1. Confirm the user has the `export_reports` permission on the workspace.                                       │
│ 2. Check recent incidents related to the `exp...                                                                │
╰────────────────────────────────── Path: docs/runbooks/export_to_csv_errors.md ──────────────────────────────────╯

╭─────────────────────────────────────────── 2. Logs Inspection Guide ────────────────────────────────────────────╮
│ Score: 5                                                                                                        │
│                                                                                                                 │
│ ...log snippet:                                                                                                 │
│                                                                                                                 │
│ 1. Look for error codes (e.g., `ERR_EXPORT_TIMEOUT`, `ERR_AUTH_INVALID_TOKEN`).                                 │
│ 2. Match error codes to known runbooks such as **Export to CSV Errors** or                                      │
│    **Authentication Failures**.                                                                                 │
│ 3. Pay attention to correlation IDs and timestamps; these may be needed if you                                  │
│    escalate to engin...                                                                                         │
╰───────────────────────────────── Path: docs/runbooks/logs_inspection_guide.md ──────────────────────────────────╯

In [10]:
# Try different queries and compare results
queries = [
    "authentication failure",
    "login sso saml",
    "workspace permissions",
    "error logs"
]

comparison_table = Table(title="Search Results Comparison")
comparison_table.add_column("Query", style="cyan")
comparison_table.add_column("Top Result", style="yellow")
comparison_table.add_column("Score", justify="right", style="magenta")
comparison_table.add_column("Total Results", justify="right", style="green")

for query in queries:
    results = await client.call_support_docs_search(query, max_results=5)
    
    if results:
        top_result = results[0]
        comparison_table.add_row(
            query,
            top_result['title'][:40] + "...",
            str(top_result['score']),
            str(len(results))
        )
    else:
        comparison_table.add_row(query, "No results", "-", "0")

console.print(comparison_table)

                             Search Results Comparison                              
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Query                  ┃ Top Result                      ┃ Score ┃ Total Results ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━┩
│ authentication failure │ Authentication Failures...      │     5 │             2 │
│ login sso saml         │ Authentication Failures...      │     7 │             1 │
│ workspace permissions  │ Workspace Permissions Issues... │    10 │             2 │
│ error logs             │ Logs Inspection Guide...        │     6 │             4 │
└────────────────────────┴─────────────────────────────────┴───────┴───────────────┘

### 🧪 Exercise: How does scoring work?

Let's test with queries of different specificity:

In [11]:
# Single word vs multi-word queries
test_queries = [
    ("export", "Single word"),
    ("export csv", "Two words"),
    ("export csv timeout", "Three words"),
    ("export csv timeout workspace", "Four words")
]

console.print("[bold cyan]Score Analysis:[/bold cyan]\n")

for query, label in test_queries:
    results = await client.call_support_docs_search(query, max_results=1)
    
    if results:
        console.print(f"[yellow]{label}:[/yellow] '{query}'")
        console.print(f"  Top match: {results[0]['title']}")
        console.print(f"  Score: [bold]{results[0]['score']}[/bold]")
        console.print()

console.print("[dim]💡 Notice: More specific queries = higher scores![/dim]")

Score Analysis:

Single word: 'export'

Top match: Export to CSV Errors

Score: 10

Two words: 'export csv'

...

Score: 15

Four words: 'export csv timeout workspace'

Top match: Export to CSV Errors

Score: 18

💡 Notice: More specific queries = higher scores!

## 3. Tool #2: incidents.search

Searches for related incidents.

### Features:
- Searches title, summary, and tags
- Can filter by incident status
- Returns matched tags for debugging

In [12]:
# Search all incidents
all_incidents = await client.call_incidents_search(
    query="export timeout",
    max_results=10  # Get all
)

console.print(f"\n[bold cyan]Found {len(all_incidents)} incidents matching 'export timeout'[/bold cyan]\n")

for incident in all_incidents:
    status_color = {
        "Investigating": "red",
        "Mitigating": "yellow",
        "Resolved": "green"
    }.get(incident['status'], "white")
    
    console.print(Panel(
        f"[{status_color}]Status: {incident['status']}[/{status_color}]\n\n"
        f"{incident['summary']}\n\n"
        f"[dim]Matched tags: {', '.join(incident['matched_tags']) if incident['matched_tags'] else 'none'}[/dim]\n"
        f"[dim]All tags: {', '.join(incident.get('tags', []))}[/dim]",
        title=f"[bold]{incident['incident_id']}: {incident['title']}[/bold]",
        border_style=status_color
    ))
    console.print()

Found 2 incidents matching 'export timeout'

╭───────────────────────────── INC-1001: Export to CSV timeouts for large workspaces ─────────────────────────────╮
│ Status: Mitigating                                                                                              │
│                                                                                                                 │
│ Some customers with very large workspaces are experiencing timeouts when exporting dashboards to CSV. The       │
│ export_service is partially degraded while we roll out a fix.                                                   │
│                                                                                                                 │
│ Matched tags: export, timeout                                                                                   │
│ All tags:                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────── INC-1004: API rate limiting spike for export endpoints ─────────────────────────────╮
│ Status: Resolved                                                                                                │
│                                                                                                                 │
│ A misconfigured rate limiting rule caused some export API calls to be throttled aggressively. The rule has been │
│ rolled back and normal behavior restored.                                                                       │
│                                                                                                                 │
│ Matched tags: export                                                                                            │
│ All tags:                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [13]:
# Filter by status - only active incidents
active_incidents = await client.call_incidents_search(
    query="export",
    max_results=10,
    status_filter=["Investigating", "Mitigating"]  # Active only
)

console.print(f"\n[bold yellow]Active incidents ({len(active_incidents)}):[/bold yellow]\n")

for incident in active_incidents:
    console.print(f"[red]●[/red] {incident['incident_id']}: {incident['title']}")
    console.print(f"  Status: {incident['status']}")
    console.print()

Active incidents (1):

● INC-1001: Export to CSV timeouts for large workspaces

Status: Mitigating

In [14]:
# Compare filtered vs unfiltered
resolved_incidents = await client.call_incidents_search(
    query="export",
    status_filter=["Resolved"]
)

console.print("[bold cyan]Incident Status Breakdown:[/bold cyan]\n")
console.print(f"  [red]Active:[/red] {len(active_incidents)}")
console.print(f"  [green]Resolved:[/green] {len(resolved_incidents)}")
console.print(f"  [white]Total:[/white] {len(active_incidents) + len(resolved_incidents)}")

Incident Status Breakdown:

Active: 1

Resolved: 1

Total: 2

### 🧪 Exercise: Tag Matching

Let's see how tag matching improves relevance:

In [15]:
# Search with tag-heavy query
tag_results = await client.call_incidents_search(
    query="sso auth authentication",  # These are tags!
    max_results=5
)

console.print("[bold cyan]Results for tag-based search:[/bold cyan]\n")

for result in tag_results:
    matched_count = len(result['matched_tags'])
    console.print(f"[yellow]{result['incident_id']}:[/yellow] {result['title']}")
    console.print(f"  Matched {matched_count} tags: {', '.join(result['matched_tags'])}")
    console.print(f"  Score: {result['score']}")
    console.print()

console.print("[dim]💡 Tag matches boost relevance![/dim]")

Results for tag-based search:

INC-1002: Authentication failures for SSO users in EU region

Matched 3 tags: auth, sso, authentication

Score: 3

💡 Tag matches boost relevance!

## 4. Tool #3: status.check

Checks current health of services.

### Design:
- Simple key-value lookup
- Returns "Unknown" for services not in the map
- Used to detect outages

In [16]:
# Check all known services
known_services = [
    "export_service",
    "auth_service",
    "workspace_service",
    "reporting_ui"
]

console.print("[bold cyan]Service Health Dashboard:[/bold cyan]\n")

status_table = Table(title="Current System Status")
status_table.add_column("Service", style="cyan")
status_table.add_column("Status", style="magenta")
status_table.add_column("Notes", style="white")

for service in known_services:
    status = await client.call_status_check(service)
    
    status_emoji = {
        "Healthy": "[green]✓[/green]",
        "Degraded": "[yellow]⚠[/yellow]",
        "Outage": "[red]✗[/red]",
        "Unknown": "[dim]?[/dim]"
    }.get(status['status'], "?")
    
    status_table.add_row(
        status['service_name'],
        f"{status_emoji} {status['status']}",
        status['notes'][:60] + "..." if len(status['notes']) > 60 else status['notes']
    )

console.print(status_table)

Service Health Dashboard:

                                       Current System Status                                        
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Service           ┃ Status     ┃ Notes                                                           ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ export_service    │ ⚠ Degraded │ Exports may time out for very large workspaces while we roll... │
│ auth_service      │ ✓ Healthy  │ No known issues. Recent SSO problems appear tied to external... │
│ workspace_service │ ✓ Healthy  │ No active incidents. A previous permissions caching bug has ... │
│ reporting_ui      │ ✓ Healthy  │ Dashboard rendering is operating normally.                      │
└───────────────────┴────────────┴─────────────────────────────────────────────────────────────────┘

In [17]:
# Test unknown service (graceful degradation)
unknown_status = await client.call_status_check("nonexistent_service")

console.print("\n[bold yellow]Testing unknown service:[/bold yellow]\n")
console.print(Panel(
    f"Service: {unknown_status['service_name']}\n"
    f"Status: {unknown_status['status']}\n"
    f"Notes: {unknown_status['notes']}",
    title="Unknown Service Response",
    border_style="yellow"
))

console.print("\n[dim]💡 The system gracefully handles unknown services![/dim]")

Testing unknown service:

╭─────────────────────────────────────────── Unknown Service Response ────────────────────────────────────────────╮
│ Service: nonexistent_service                                                                                    │
│ Status: Unknown                                                                                                 │
│ Notes: Service 'nonexistent_service' not found in status map.                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

💡 The system gracefully handles unknown services!

## 5. Understanding Tool Outputs

Let's examine the actual JSON returned by each tool:

In [18]:
# Get raw JSON output
doc_result = await client.call_support_docs_search("export", max_results=1)

console.print("[bold cyan]Docs Tool Output Structure:[/bold cyan]\n")
syntax = Syntax(json.dumps(doc_result, indent=2), "json", theme="monokai")
console.print(syntax)

Docs Tool Output Structure:

[                                                                                                                  
  {                                                                                                                
    "title": "Export to CSV Errors",                                                                               
    "snippet": "# Export to CSV Errors\n\nWhen users attempt to export reports to CSV and see errors like\n**\"Expo
    "path": "docs/runbooks/export_to_csv_errors.md",                                                               
    "score": 10                                                                                                    
  }                                                                                                                
]                                                                                                                  

In [19]:
# Incident tool output
incident_result = await client.call_incidents_search("export", max_results=1)

console.print("\n[bold cyan]Incidents Tool Output Structure:[/bold cyan]\n")
syntax = Syntax(json.dumps(incident_result, indent=2), "json", theme="monokai")
console.print(syntax)

Incidents Tool Output Structure:

[                                                                                                                  
  {                                                                                                                
    "incident_id": "INC-1001",                                                                                     
    "title": "Export to CSV timeouts for large workspaces",                                                        
    "status": "Mitigating",                                                                                        
    "summary": "Some customers with very large workspaces are experiencing timeouts when exporting dashboards to CS
    "matched_tags": [                                                                                              
      "export"                                                                                                     
    ],                                                                                                             
    "score": 1                                                                                                     
  }                                                                                                                
]                                                                                                                  

In [20]:
# Status tool output
status_result = await client.call_status_check("export_service")

console.print("\n[bold cyan]Status Tool Output Structure:[/bold cyan]\n")
syntax = Syntax(json.dumps(status_result, indent=2), "json", theme="monokai")
console.print(syntax)

Status Tool Output Structure:

{                                                                                                                  
  "service_name": "export_service",                                                                                
  "status": "Degraded",                                                                                            
  "notes": "Exports may time out for very large workspaces while we roll out performance improvements."            
}                                                                                                                  

## 6. Performance Analysis

Let's measure how fast each tool is:

In [21]:
import time

async def time_tool_call(name, coro):
    start = time.time()
    result = await coro
    elapsed = (time.time() - start) * 1000  # Convert to ms
    return name, elapsed, len(result) if isinstance(result, list) else 1

# Time each tool
timings = [
    await time_tool_call("support_docs.search", 
                         client.call_support_docs_search("export csv timeout", 3)),
    await time_tool_call("incidents.search", 
                         client.call_incidents_search("export timeout", 3)),
    await time_tool_call("status.check", 
                         client.call_status_check("export_service"))
]

# Display results
timing_table = Table(title="Tool Performance")
timing_table.add_column("Tool", style="cyan")
timing_table.add_column("Time (ms)", justify="right", style="yellow")
timing_table.add_column("Results", justify="right", style="green")

for name, ms, count in timings:
    timing_table.add_row(name, f"{ms:.2f}", str(count))

console.print(timing_table)
console.print("\n[dim]💡 All tools are fast because data is in memory![/dim]")

              Tool Performance               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┓
┃ Tool                ┃ Time (ms) ┃ Results ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━┩
│ support_docs.search │      1.95 │       2 │
│ incidents.search    │      1.73 │       2 │
│ status.check        │      1.21 │       1 │
└─────────────────────┴───────────┴─────────┘

💡 All tools are fast because data is in memory!

## 7. Real-World Scenario: Ticket Triage

Let's simulate what happens when a ticket comes in:

In [22]:
# Simulated ticket
ticket_description = "Customer reports export to CSV is failing with timeout error after 30 seconds"
ticket_logs = "ERR_EXPORT_TIMEOUT service=export_service workspace=ws_1234"

console.print("[bold magenta]Incoming Ticket:[/bold magenta]\n")
console.print(Panel(
    f"[yellow]Description:[/yellow]\n{ticket_description}\n\n"
    f"[yellow]Logs:[/yellow]\n{ticket_logs}",
    title="TICKET-999",
    border_style="blue"
))

# Extract keywords for search
search_query = "export csv timeout"
console.print(f"\n[cyan]Search query: '{search_query}'[/cyan]\n")

Incoming Ticket:

╭────────────────────────────────────────────────── TICKET-999 ───────────────────────────────────────────────────╮
│ Description:                                                                                                    │
│ Customer reports export to CSV is failing with timeout error after 30 seconds                                   │
│                                                                                                                 │
│ Logs:                                                                                                           │
│ ERR_EXPORT_TIMEOUT service=export_service workspace=ws_1234                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Search query: 'export csv timeout'

In [23]:
# Step 1: Search docs
console.print("[bold]Step 1: Searching documentation...[/bold]")
docs = await client.call_support_docs_search(search_query, 2)

if docs:
    console.print(f"[green]✓ Found {len(docs)} relevant runbooks[/green]")
    for doc in docs:
        console.print(f"  • {doc['title']} (score: {doc['score']})")
else:
    console.print("[yellow]No documentation found[/yellow]")

console.print()

Step 1: Searching documentation...

✓ Found 2 relevant runbooks

• Export to CSV Errors (score: 15)

• Logs Inspection Guide (score: 5)

In [24]:
# Step 2: Search incidents
console.print("[bold]Step 2: Searching for related incidents...[/bold]")
incidents = await client.call_incidents_search(
    search_query, 
    max_results=2,
    status_filter=["Investigating", "Mitigating"]  # Active only
)

if incidents:
    console.print(f"[yellow]⚠ Found {len(incidents)} active incidents[/yellow]")
    for inc in incidents:
        console.print(f"  • {inc['incident_id']}: {inc['title']} ({inc['status']})")
else:
    console.print("[green]No active incidents found[/green]")

console.print()

Step 2: Searching for related incidents...

⚠ Found 1 active incidents

• INC-1001: Export to CSV timeouts for large workspaces (Mitigating)

In [25]:
# Step 3: Check service status
console.print("[bold]Step 3: Checking service health...[/bold]")
status = await client.call_status_check("export_service")

status_color = {
    "Healthy": "green",
    "Degraded": "yellow",
    "Outage": "red"
}.get(status['status'], "white")

console.print(f"[{status_color}]{status['service_name']}: {status['status']}[/{status_color}]")
console.print(f"[dim]{status['notes']}[/dim]")
console.print()

Step 3: Checking service health...

export_service: Degraded

Exports may time out for very large workspaces while we roll out performance improvements.

In [26]:
# Summary of findings
console.print("[bold cyan]Summary of Research:[/bold cyan]\n")
console.print(f"[green]✓[/green] Found runbook: {docs[0]['title'] if docs else 'None'}")
console.print(f"[yellow]⚠[/yellow] Active incident: {incidents[0]['incident_id'] if incidents else 'None'}")
console.print(f"[{status_color}]●[/{status_color}] Service status: {status['status']}")
console.print()
console.print("[bold]Recommended Action:[/bold]")
if status['status'] == 'Degraded' and incidents:
    console.print("[yellow]This is a known issue. Follow the runbook and mention the ongoing incident.[/yellow]")
elif status['status'] == 'Healthy' and not incidents:
    console.print("[green]Isolated issue. Follow the runbook for troubleshooting.[/green]")
else:
    console.print("[red]Potential new issue. Escalate to engineering.[/red]")

Summary of Research:

✓ Found runbook: Export to CSV Errors

⚠ Active incident: INC-1001

● Service status: Degraded

Recommended Action:

This is a known issue. Follow the runbook and mention the ongoing incident.

## 8. Cleanup

In [27]:
# Disconnect from MCP server
await client.disconnect()
console.print("[green]✓ Disconnected from MCP server[/green]")

RuntimeError: Attempted to exit cancel scope in a different task than it was entered in

---

## Summary

In this notebook, we explored:

1. ✅ **What MCP is** and why it's useful
2. ✅ **How to connect** to an MCP server
3. ✅ **All 3 tools** in detail:
   - `support_docs.search` - Keyword search with scoring
   - `incidents.search` - Tag-aware search with filtering
   - `status.check` - Simple key-value lookup
4. ✅ **Search behavior** - How different queries affect results
5. ✅ **Performance** - All tools are fast (in-memory data)
6. ✅ **Real workflow** - How tools combine to triage a ticket

## Key Insights

- **MCP standardizes tool integration** - No custom protocols
- **Simple algorithms work well** for small datasets
- **Tags improve relevance** in incident search
- **Graceful degradation** handles edge cases (unknown services)
- **Tools complement each other** - docs + incidents + status = complete picture

## Next Steps

- Explore how agents **use these tools** in `demo_complete_flow.ipynb`
- Try **modifying search algorithms** in `support_mcp_server/tools_*.py`
- **Add new tools** following the same pattern

---

**Happy exploring!** 🔧